In [7]:
pip install --user google-cloud-aiplatform --upgrade

E1127 17:36:31.448090889     110 backup_poller.cc:127]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2023-11-27T17:36:31.447867087+00:00", children:[UNKNOWN:Bad file descriptor {created_time:"2023-11-27T17:36:31.447793305+00:00", errno:9, os_error:"Bad file descriptor", syscall:"epoll_wait"}]}
  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/20/ee/af1e3c9cb2bed3aa91297f8362da3879a793d2555c2e587a9c3d2f1ed900/google_cloud_aiplatform-1.34.0-py2.py3-none-any.whl.metadata
  Using cached google_cloud_aiplatform-1.34.0-py2.py3-none-any.whl.metadata (28 kB)
Using cached google_cloud_aiplatform-1.34.0-py2.py3-none-any.whl (3.1 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.30.1
    Uninstalling google-cloud-aiplatform-1.30.1:
      Successfully uninstalled google-cloud-aiplatform-1.30.1
  Consider adding this directory to PATH or, if 

In [1]:
!python3 --version

Python 3.7.12


### NOTE: Restart your kernel now so that you can use installed packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import aiplatform
import vertexai
from vertexai.language_models import CodeChatModel

In [2]:
#inititate vertex code generation model
vertexai.init(project="solutions-2023-mar-107", location="us-central1")
chat_model = CodeChatModel.from_pretrained("codechat-bison")
chat = chat_model.start_chat()

In [6]:
aiplatform.__version__

'1.30.1'

In [3]:
%%bigquery df
SELECT * FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE store_name = "WAL-MART 3394 / ATLANTIC"
AND date >= date("2023-01-01")

Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
pd.set_option('display.max_columns', None)
df_head = df.head(5)
df_head

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-60753500003,2023-07-28,3833,WAL-MART 3394 / ATLANTIC,1905 EAST 7TH ST,ATLANTIC,50022.0,POINT(-94.985986964 41.401588009),None,CASS,1012100.0,CANADIAN WHISKIES,434,LUXCO INC,10935,HAWKEYE CANADIAN WHISKY,6,1750,9.50,14.25,6,85.50,10.5,2.77
1,INV-57955300025,2023-04-28,3833,WAL-MART 3394 / ATLANTIC,1905 EAST 7TH ST,ATLANTIC,50022.0,POINT(-94.9859869639999 41.4015880090001),None,CASS,1031100.0,AMERICAN VODKAS,260,DIAGEO AMERICAS,37998,SMIRNOFF 80PRF,6,1750,14.75,22.13,6,132.78,10.5,2.77
2,INV-57114700013,2023-03-31,3833,WAL-MART 3394 / ATLANTIC,1905 EAST 7TH ST,ATLANTIC,50022,POINT(-94.9873 41.40387),None,CASS,1032100.0,IMPORTED VODKAS,370,PERNOD RICARD USA,34004,ABSOLUT SWEDISH VODKA 80PRF,24,375,4.99,7.49,24,179.76,9.0,2.37
3,INV-61597400020,2023-08-25,3833,WAL-MART 3394 / ATLANTIC,1905 EAST 7TH ST,ATLANTIC,50022.0,POINT(-94.985986964 41.401588009),None,CASS,1031100.0,AMERICAN VODKAS,421,SAZERAC COMPANY INC,35944,FLEISCHMANNS 80PRF VODKA,24,375,1.74,2.61,24,62.64,9.0,2.37
4,INV-59045500013,2023-06-03,3833,WAL-MART 3394 / ATLANTIC,1905 EAST 7TH ST,ATLANTIC,50022.0,POINT(-94.985986964 41.401588009),None,CASS,1042100.0,IMPORTED DRY GINS,260,DIAGEO AMERICAS,28868,TANQUERAY GIN,6,1750,26.49,39.74,6,238.44,10.5,2.77


In [5]:
df_head = df.head(10)
schema = df.dtypes

PREFIX = """
        Instructions:
        You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
        Your job is to generate python code based on the questions asked by the user.
        This code should be ready to be executed inside a python environment therefore you do not need to include the "python" keyword.
        Please finish the code with a print statement or a plot of the results

        You can use the following libraries:
        - pandas
        - numpy
        - matplotlib

        This is the schema of the dataframe:
        {schema}
        This is the result of `print(df.head())`:
        {df_head}
        Begin!
        Question:""".format(df_head=df_head, schema=schema)

SUFFIX = """
        Answer:"""


def generate_pandas_code(prompt, prefix=PREFIX, suffix=SUFFIX ):

  parameters = {
      "candidate_count": 1,
      "max_output_tokens": 1024,
      "temperature": 0.2
  }

  prompt = prefix + prompt + suffix

  response = chat.send_message(prompt, **parameters).text
  print(response)
  # delete first line of the response
  edit_res = response.replace(response.split('\n')[0]+"\n", '')
  # delete last line of the response
  edit_res = edit_res.replace(edit_res.split('\n')[-1], '')

  return edit_res


def execute_code(code):
  try:
    print("Excecuting code...")
    print(eval(code))
    print("Done excecuting code...")
  except Exception as e:
    print(e)

In [10]:
prompt = "How many bottles were sold?"
execute_code(generate_pandas_code(prompt))

TypeError: send_message() got an unexpected keyword argument 'candidate_count'